In [1]:
import pyfits
import numpy as np
import matplotlib as plt
from pylab import *
from astropy.cosmology import WMAP9 as cosmo
from matplotlib.colors import LogNorm

import scipy


import heapq
import pyfits
import matplotlib as plt
from pylab import *
import numpy as np
from decimal import *
import numpy
from scipy.optimize import curve_fit
from scipy.integrate import quad
import matplotlib.pyplot as plt
from astropy.modeling import models, fitting
from pylab import *
from scipy import stats
from scipy.stats import chisquare
from astropy.io import fits
import copy



In [2]:
ins='Figs/kin_class_all_clip.txt'
print(ins.split('.txt')[0].split('kin_class_')[1])
add_on=ins.split('.txt')[0].split('kin_class_')[1]


all_clip


In [3]:
file_new = open('Figs/kin_table_'+str(add_on)+'.txt','w')
i=0

with open(ins, 'r') as f:
    data = f.readlines()
    for line in data:
        
        words_i = line.split()
        
        
        print(str(words_i[0]))
        if words_i[0]=='&':
            '''This is for reading two rows together'''
            '''First, determine how many Gaussian components is best'''
 
            if float(words[i][3].split('/')[0])/float(words[i][3].split('/')[1]) > 0.5 or float(words[i-1][4].split('/')[0])/float(words[i-1][4].split('/')[1]) > 0.5:
                num_gauss='$> 2$'
            else:
                num_gauss='2'
            #If its odd in length that means the divide is in the middle
            
            '''Next determine if velocity is above the threshhold value'''
            
            vel_before=float(words[i-1][6].split('\substack{+')[0].split('$')[1])
            if vel_before > 0:
                vel_before_plus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_minus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_before_minus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_plus=float(words[i-1][6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            vel=float(words[i][5].split('\substack{+')[0].split('$')[1])
            if vel > 0:
                vel_plus=float(words[i][5].split('\substack{+')[1].split('\\\-')[0])
                vel_minus=float(words[i][5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_minus=float(words[i][5].split('\substack{+')[1].split('\\\-')[0])
                vel_plus=float(words[i][5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
              
            if abs(vel) > 400 or abs(vel_before) > 400:
                '''Now were in the greater than or equal to regime'''
                
            
                if (abs(vel_before)-abs(vel_before_minus)) > 400 or (abs(vel) - vel_minus) > 400:
                    vel_cutoff='$> 400$'
                else:
                    vel_cutoff='$\geq 400$'
                    
            else:
                if abs(vel) < 400 and abs(vel_before) < 400:
                    if (abs(vel_before)+abs(vel_before_plus)) < 400 and (abs(vel)+abs(vel_plus)) < 400:
                        vel_cutoff='$< 400$'
                    else:
                        if (abs(vel_before)+abs(vel_before_plus)) > 400 or (abs(vel)+abs(vel_plus)) > 400:
                            vel_cutoff='$\leq 400$'
                        else:
                            vel_cutoff=':/'
                else:
                    vel_cutoff=':/'
            '''Disp overall now'''
            
            sig_overall=float(words[i][7].split('\substack{+')[0].split('$')[1])
            sig_plus=float(words[i][7].split('\substack{+')[1].split('\\\-')[0])
            sig_minus=float(words[i][7].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            
            sig_prev_overall=float(words[i-1][8].split('\substack{+')[0].split('$')[1])
            sig_prev_plus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[0])
            sig_prev_minus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            if sig_overall > 500 or sig_prev_overall > 500:
                if sig_overall - sig_minus < 500 and sig_prev_overall - sig_prev_minus < 500: 
                    sig_o='$\geq 500 $'
                else:
                    if sig_overall - sig_minus > 500 or sig_prev_overall - sig_prev_minus > 500:
                        sig_o='$> 500$'
                    else:
                        sig_o=':/'
            else:
                if sig_overall < 500 and sig_prev_overall < 500:
                    if sig_overall + sig_plus > 500 or sig_prev_overall + sig_prev_plus > 500:
                        sig_o='$\leq 500 $'
                    else:
                        if sig_overall + sig_plus < 500 and sig_prev_overall + sig_prev_plus < 500:
                            sig_o='$< 500$'
                        else:
                            sig_o=':/'
                else:
                    sig_o=':/'
                    
                    
            
            
            '''Doing the hardest part now with disperson of all four of the individual components'''
            
            #print(words[i][11].split('\substack{+')[0].split('$')[1])
            sig_1=float(words[i-1][8].split('\substack{+')[0].split('$')[1])
            sig_1_minus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_1_plus=float(words[i-1][8].split('\substack{+')[1].split('\\\-')[0])
            
            sig_2=float(words[i-1][10].split('\substack{+')[0].split('$')[1])
            sig_2_minus=float(words[i-1][10].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_2_plus=float(words[i-1][10].split('\substack{+')[1].split('\\\-')[0])
            
            sig_3=float(words[i][9].split('\substack{+')[0].split('$')[1])
            sig_3_minus=float(words[i][9].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_3_plus=float(words[i][9].split('\substack{+')[1].split('\\\-')[0])
            
            sig_4=float(words[i][11].split('\substack{+')[0].split('$')[1])
            sig_4_minus=float(words[i][11].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_4_plus=float(words[i][11].split('\substack{+')[1].split('\\\-')[0])
            
            
            
            if sig_1 > 500 or sig_2 > 500 or sig_3 > 500 or sig_4 > 500:
                '''This means were an outflow!'''
                if sig_1 - sig_1_minus > 500 or sig_2 - sig_2_minus > 500 or sig_3 - sig_3_minus > 500 or sig_4 - sig_4_minus > 500:
                    
                    sig_mult='$> 500$'
                else:
                    if sig_1 - sig_1_minus < 500 and sig_2 - sig_2_minus < 500 and sig_3 - sig_3_minus < 500 and sig_4 - sig_4_minus < 500:
                        sig_mult='$/geq 500$'
                    else:
                        sig_mult=':/'
            else:
                if sig_1 < 500 and sig_2 < 500 and sig_3 < 500 and sig_4 < 500:
                    if sig_1 + sig_1_plus > 500 or sig_2 + sig_2_plus > 500 or sig_3 + sig_3_plus > 500 or sig_4 + sig_4_plus > 500:
                        sig_mult='$/leq 500$'
                    else:
                        
                        if sig_1 + sig_1_plus < 500 and sig_2 + sig_2_plus < 500 and sig_3 + sig_3_plus < 500 and sig_4 + sig_4_plus < 500:
                            sig_mult='$< 500$'
                        else:
                            sig_mult=':/'
                else:
                    sig_mult=':/'
                    
            
            PA_gal=float(words[i-1][14])
            PA_OIII=float(words[i-1][16].split('$')[1])
            '''Now determine if a galaxy is asymmetric or not'''
            A_2=abs(float(words[i][18]))
            
            
            
            A_1=abs(float(words[i-1][18]))
            print('A_1 and A_2', A_1, A_2)
                    
            
            
            if abs(PA_gal-PA_OIII) > 20:
                al='no'
            else:
                al='yes'
                
            '''Classification Time'''
            if sig_mult=='$> 500$' or sig_mult=='$/geq 500$':
                if num_gauss=='$> 2$':
                    classs='Outflow Composite'
                if num_gauss=='2':
                    classs='Outflow'
            else:
                if vel_cutoff=='$\geq 400$' or vel_cutoff=='$> 400$':
                    if num_gauss=='$> 2$':
                        classs='Outflow Composite'
                    if num_gauss=='2':
                        classs='Outflow'
                else:
                    if al=='no':
                        classs='Ambiguous'
                    if al=='yes':
                        classs='Rotation Dominated'
                
            file_new.write(str(words_prev[0])+' & '+str(num_gauss)+' & '+str(vel_cutoff)+
                           ' & '+str(sig_o)+' & '+str(sig_mult)+' & '+str(al)+' & '+str(classs)+
                           '\\\ '+'\n')
        i+=1
        words_prev=words_i
        
file_new.close()
print('done')

J0006+1548
&


NameError: name 'words' is not defined

In [4]:
file_new = open('Figs/kin_table_'+str(add_on)+'.txt','w')
i=0

with open(ins, 'r') as f:
    data = f.readlines()
    for line in data:
        
        words = line.split()
        
        
        
        print(str(words[0]))
        
        if words[0]=='&':
            '''This is for reading two rows together'''
            '''First, determine how many Gaussian components is best'''
 
            if float(words[3].split('/')[0])/float(words[3].split('/')[1]) > 0.5 or float(words_prev[4].split('/')[0])/float(words_prev[4].split('/')[1]) > 0.5:
                num_gauss='$> 2$'
            else:
                num_gauss='2'
            #If its odd in length that means the divide is in the middle
            
            '''Next determine if velocity is above the threshhold value'''
            
            vel_before=float(words_prev[6].split('\substack{+')[0].split('$')[1])
            if vel_before > 0:
                vel_before_plus=float(words_prev[6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_minus=float(words_prev[6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_before_minus=float(words_prev[6].split('\substack{+')[1].split('\\\-')[0])
                vel_before_plus=float(words_prev[6].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            vel=float(words[5].split('\substack{+')[0].split('$')[1])
            if vel > 0:
                vel_plus=float(words[5].split('\substack{+')[1].split('\\\-')[0])
                vel_minus=float(words[5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            else:
                vel_minus=float(words[5].split('\substack{+')[1].split('\\\-')[0])
                vel_plus=float(words[5].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
              
            if abs(vel) > 400 or abs(vel_before) > 400:
                '''Now were in the greater than or equal to regime'''
                
            
                if (abs(vel_before)-abs(vel_before_minus)) > 400 or (abs(vel) - vel_minus) > 400:
                    vel_cutoff='$> 400$'
                else:
                    vel_cutoff='$\geq 400$'
                    
            else:
                if abs(vel) < 400 and abs(vel_before) < 400:
                    if (abs(vel_before)+abs(vel_before_plus)) < 400 and (abs(vel)+abs(vel_plus)) < 400:
                        vel_cutoff='$< 400$'
                    else:
                        if (abs(vel_before)+abs(vel_before_plus)) > 400 or (abs(vel)+abs(vel_plus)) > 400:
                            vel_cutoff='$\leq 400$'
                        else:
                            vel_cutoff=':/'
                else:
                    vel_cutoff=':/'
            '''Disp overall now'''
            
            sig_overall=float(words[7].split('\substack{+')[0].split('$')[1])
            sig_plus=float(words[7].split('\substack{+')[1].split('\\\-')[0])
            sig_minus=float(words[7].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            
            sig_prev_overall=float(words_prev[8].split('\substack{+')[0].split('$')[1])
            sig_prev_plus=float(words_prev[8].split('\substack{+')[1].split('\\\-')[0])
            sig_prev_minus=float(words_prev[8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            
            if sig_overall > 500 or sig_prev_overall > 500:
                if sig_overall - sig_minus < 500 and sig_prev_overall - sig_prev_minus < 500: 
                    sig_o='$\geq 500 $'
                else:
                    if sig_overall - sig_minus > 500 or sig_prev_overall - sig_prev_minus > 500:
                        sig_o='$> 500$'
                    else:
                        sig_o=':/'
            else:
                if sig_overall < 500 and sig_prev_overall < 500:
                    if sig_overall + sig_plus > 500 or sig_prev_overall + sig_prev_plus > 500:
                        sig_o='$\leq 500 $'
                    else:
                        if sig_overall + sig_plus < 500 and sig_prev_overall + sig_prev_plus < 500:
                            sig_o='$< 500$'
                        else:
                            sig_o=':/'
                else:
                    sig_o=':/'
                    
                    
            
            
            '''Doing the hardest part now with disperson of all four of the individual components'''
            
            #print(words[i][11].split('\substack{+')[0].split('$')[1])
            sig_1=float(words_prev[8].split('\substack{+')[0].split('$')[1])
            sig_1_minus=float(words_prev[8].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_1_plus=float(words_prev[8].split('\substack{+')[1].split('\\\-')[0])
            
            sig_2=float(words_prev[10].split('\substack{+')[0].split('$')[1])
            sig_2_minus=float(words_prev[10].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_2_plus=float(words_prev[10].split('\substack{+')[1].split('\\\-')[0])
            
            sig_3=float(words[9].split('\substack{+')[0].split('$')[1])
            sig_3_minus=float(words[9].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_3_plus=float(words[9].split('\substack{+')[1].split('\\\-')[0])
            
            sig_4=float(words[11].split('\substack{+')[0].split('$')[1])
            sig_4_minus=float(words[11].split('\substack{+')[1].split('\\\-')[1].split('}$')[0])
            sig_4_plus=float(words[11].split('\substack{+')[1].split('\\\-')[0])
            
            
            
            if sig_1 > 500 or sig_2 > 500 or sig_3 > 500 or sig_4 > 500:
                '''This means were an outflow!'''
                if sig_1 - sig_1_minus > 500 or sig_2 - sig_2_minus > 500 or sig_3 - sig_3_minus > 500 or sig_4 - sig_4_minus > 500:
                    
                    sig_mult='$> 500$'
                else:
                    if sig_1 - sig_1_minus < 500 and sig_2 - sig_2_minus < 500 and sig_3 - sig_3_minus < 500 and sig_4 - sig_4_minus < 500:
                        sig_mult='$\geq 500$'
                    else:
                        sig_mult=':/'
            else:
                if sig_1 < 500 and sig_2 < 500 and sig_3 < 500 and sig_4 < 500:
                    if sig_1 + sig_1_plus > 500 or sig_2 + sig_2_plus > 500 or sig_3 + sig_3_plus > 500 or sig_4 + sig_4_plus > 500:
                        sig_mult='$\leq 500$'
                    else:
                        
                        if sig_1 + sig_1_plus < 500 and sig_2 + sig_2_plus < 500 and sig_3 + sig_3_plus < 500 and sig_4 + sig_4_plus < 500:
                            sig_mult='$< 500$'
                        else:
                            sig_mult=':/'
                else:
                    sig_mult=':/'
                    
            
            PA_gal=float(words_prev[14])
            PA_OIII=float(words_prev[16].split('$')[1])
            
            if abs(PA_gal-PA_OIII) > 20:
                al='no'
            else:
                al='yes'
            
            
            A_1=abs(float(words_prev[20]))
            A_2=abs(float(words[15]))
            print('A_1 and A_2', A_1, A_2)
            '''The limits are the interval: 0.195894695446 to 0.341684251922'''
            '''Compare to the maximum value of A_1 or A_2'''
            if A_1< 0.1958 and A_2 < 0.1958:
                '''the galaxy is symmetry'''
                assy='no'
            else:
                assy='yes'
                
            '''Classification Time'''
            if sig_mult=='$> 500$' or sig_mult=='$\geq 500$':
                if num_gauss=='$> 2$':
                    classs='Outflow Composite'
                if num_gauss=='2':
                    classs='Outflow'
            else:
                if vel_cutoff=='$\geq 400$' or vel_cutoff=='$> 400$':
                    if num_gauss=='$> 2$':
                        classs='Outflow Composite'
                    if num_gauss=='2':
                        classs='Outflow'
                else:
                    if al=='no':
                        classs='Ambiguous'
                    if al=='yes':
                        if assy=='no':
                            classs='Rotation Dominated + Obscuration'
                        if assy=='yes':
                            classs='Rotation Dominated + Disturbance'
            print('class', classs)  
            file_new.write(str(words_prev[0])+' & '+str(num_gauss)+' & '+str(vel_cutoff)+
                           ' & '+str(sig_o)+' & '+str(sig_mult)+' & '+str(al)+' & '+str(classs)+
                           '\\\ '+'\n')
        i+=1
        words_prev=words
        
file_new.close()
print('done')

J0006+1548
&
A_1 and A_2 0.08 0.11
class Ambiguous
J0116-1025
&
A_1 and A_2 0.27 0.19
class Rotation Dominated + Disturbance
J0118-0826
&
A_1 and A_2 0.18 0.06
class Rotation Dominated + Obscuration
J0135-0058
&
A_1 and A_2 0.15 0.23
class Rotation Dominated + Disturbance
J0142-0050
&
A_1 and A_2 0.22 0.04
class Ambiguous
J0200-0931
&
A_1 and A_2 0.09 0.07
class Outflow Composite
J0400-0652
&
A_1 and A_2 0.14 0.7
class Rotation Dominated + Disturbance
J0735+4036
&
A_1 and A_2 0.27 0.13
class Outflow Composite
J0741+3928
&
A_1 and A_2 0.37 0.11
class Outflow Composite
J0747+3440
&
A_1 and A_2 0.1 0.09
class Outflow
J0804+3051
&
A_1 and A_2 0.17 0.1
class Outflow Composite
J0808+4813
&
A_1 and A_2 0.12 0.06
class Ambiguous
J0815+0635
&
A_1 and A_2 0.11 0.21
class Rotation Dominated + Disturbance
J0840+2727
&
A_1 and A_2 0.18 0.14
class Outflow
J0841+3931
&
A_1 and A_2 0.06 0.1
class Ambiguous
J0846+4258
&
A_1 and A_2 0.15 0.11
class Outflow
J0858+1041
&
A_1 and A_2 0.15 0.13
class Ambigu